#### The pipeline to analyse Asteraceae genomes using Asteraceae synteny-phylogenomic framework

In [2]:
# get dependencies ready
import os
from pathlib import Path
from utils import processLCSAndFirstFilter_2sp as plff
from utils import processFinalFilter_2sp as pff
import shutil
from processDrimm_tao_2sp import readSequence,prepare_dir,write_raw_blocks,processDrimm
from pathlib import Path
import pandas as pd
import numpy as np
from modules_paper import prepare_OG,prepare_meta,make_comBed,combed_parser,grimmBlock_parser,getFilterSequence,getAllSequence,outSequence,standarlize_drimmblocks,get_block_info,get_chromosome_info
from parse_genespaceBED_for_drimmBED_paper import make_index,make_drimmBED,make_genome_meta,make_chromosome_meta,get_chromosome_lenght,parse_genespaceBED_for_drimmBED

In [2]:
# get input data ready
sp='sp46'
index_file = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/Smar.EM05.v1.genome.fa.fai"

workdir = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir"
BED_folder = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/"
if not workdir[-1] == "/": workdir += "/"
if not BED_folder[-1] == "/": BED_folder += "/"

In [ ]:
# step0, first let's prepare bed and meta data
## make index file
df_index = make_index(sp,index_file) #your genome index and genome name you want to use

## make bed, genome_meta, chromosome_meta
DIC = {f"{row['sp']}@{row['chr']}": row['length'] for _, row in df_index.iterrows()}
bed_dir = workdir + "bed/"
if not os.path.exists(bed_dir):
    os.makedirs(bed_dir)
    make_drimmBED(BED_folder, DIC, bed_dir)
else:
    print(bed_dir + " exists, let's generate bed file in this floder")
    make_drimmBED(BED_folder, DIC, bed_dir)

In [ ]:
# step1 process synOG
## get dependent info ready, read in synOG table
from processOrthofinder_tao import get_group_dir,get_final_group,main

## read in synOG table and adjust name
orthogroups = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/2_analysis/1_genespace/run30/pangenome_sp03v5_synOG_2sp_syn.txt"
## be careful with this orthogroups table; in terms of the name "sp03v5" vs ""sp03_v5"", and "sp03v5@gene1" vs "sp03v5_gene1"; we need to make this consistent in genespace_parser()
ortho = pd.read_csv(orthogroups, sep='\t', dtype = str, keep_default_na=False)
ortho2 = ortho[ortho["interpChr"] != ""]

ortho2.rename(columns={'sp03_v5': 'sp03v5', 'sp03_v1': 'sp03v1'}, inplace=True)
ortho2['sp03v5'] = ortho2['sp03v5'].str.replace('sp03_v5@', 'sp03v5@')
ortho2.head()

In [ ]:
## process synOG table, call pairwise orthologs for given ref-target pair
##target = ["sp28","sp24","sp43","sp44","sp18","sp34","sp14","sp10","sp08","sp35","sp05a","sp39","sp37","sp11","sp13","sp22","sp21","sp29","sp30","sp03_v5","sp03_v1","sp03","sp41","sp07","sp15"]
target = ["sp46"]
reference = "sp03v5"
OG_type = "synOG" # always check the varable "sp_ratio"

genome_meta = make_genome_meta("sp46",1,17)
chromosome_meta = make_chromosome_meta(BED_folder,DIC)
print(chromosome_meta)

outdir = workdir + "out1"
if outdir[-1] != "/": outdir += "/"
if bed_dir[-1] != "/": bed_dir += "/"

for species in target:
    if not species == reference: 
        species_list = [reference, species]
        subdir = outdir + species + "/1_SynOG/"
        if not Path(subdir).exists():
            os.makedirs(subdir)
            sp,sp_ratio,sp_chr_number,long_chr_list,gff_list = prepare_meta(species_list,genome_meta,chromosome_meta,OG_type,bed_dir)
            #print(sp)
            #print(sp_chr_number)
            print(gff_list,long_chr_list)
            ortho3 = prepare_OG(ortho2,sp,OG_type)
            group_dir = get_group_dir(ortho3,sp)
            finalGroup = get_final_group(group_dir,sp_ratio)
            main(bed_dir,subdir,group_dir,finalGroup,gff_list,long_chr_list)
        else: print(species + " seems already done, please check!")

### Step2 perform drimm-synteny in windows
1. transfer the data from last step to windows
   #put the whole directory (e.g., genespace26) in C:\Users\fengtao\Desktop\drimm_batch\
2. double click the windows bat file "drimm-synteny"; the tool will loop the genomes in "genespace26" and perform calculation
   
   **input poped options:**
     + The path of drimm.sequence <drimm.sequence>
     + The output directory <.>
     + cycleLengthThreshold controls the continuity of synteny blocks (default parameter is 20)
     + dustThreshold controls the upper limit of gene family. The gene family will be filtered when homologous genes exceeding dustThreshold. For above example, target copy numbers are 2,4,2,2,2 in each species, then the dustThreshold is 13 (2+4+2+2+2+1)
3. wait untill the calculations done, copy the results to MAC, and go ahead with next step

A  ***drimm-synteny.bat*** file is like this:
>
> @echo off
>
> REM Loop through each subdirectory
> for /d %%A in ("C:\Users\fengtao\Desktop\drimm_batch\genespace26\*") do (
   >
    > echo Entering directory: %%A\2_DrimmRaw\
    >
    > mkdir "%%A\2_DrimmRaw\"
    >
    > copy "%%A\1_SynOG\drimm.sequence" "%%A\2_DrimmRaw\"
    >
    > pushd "%%A\2_DrimmRaw\"
    >
    >
    > echo inputs like this, modify accordingly: inputfile=drimm.sequence  outdir=.  cycleLengthThreshold=20  dustThreshold=5
    >
    >
    > REM Run your executable tool with non-interactive inputs
    >
    > "C:\Users\fengtao\Desktop\processDrimm\drimm\drimm\bin\Release\netcoreapp3.1\win-x64\drimm.exe"
    >
    > echo %%A done!
    >
    > popd
> )
> pause

In [ ]:
# step3 process drimm
# from drimm-synteny, we get raw blocks, and we need to clean and sort the raw blcoks to
## 1. split the all-in-one blocks (drimm-synteny output) into individual genomes
## 2. meanwhile divide the blocks into separate ones based on ratio (normally from genome duplication, but segmental duplications cannot be distinguished, and are also processed)
workdir = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir"
BED_folder = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/"
if workdir[-1] != "/": workdir += "/"
if BED_folder[-1] != "/": BED_folder += "/"

outdir = workdir + "out1/"

sp='sp46'
index_file = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/Smar.EM05.v1.genome.fa.fai"
df_index = make_index(sp,index_file) #your genome index and genome name you want to use
DIC = {f"{row['sp']}@{row['chr']}": row['length'] for _, row in df_index.iterrows()}

genome_meta = make_genome_meta("sp46",1,17)
chromosome_meta = make_chromosome_meta(BED_folder,DIC)

#bed_path = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/2_analysis/10_ancestralGenome/bed"
OG_type = "synOG" # synOG|orthofinder
working_dir = outdir
bed_dir = workdir + "bed/"

if working_dir[-1] != "/": working_dir += "/"
if bed_dir[-1] != "/": bed_dir += "/"

target = ["sp46"]
reference = "sp03v5"

for species in target:
    if not species == reference: 
        species_list = [reference, species]
        sp_working_dir = working_dir + species + "/"
        print(sp_working_dir)
        homolog_dir = sp_working_dir + "1_SynOG/"
        if not Path(sp_working_dir).exists():
            print("no homologs found for " + species)
        else:
            block_file = sp_working_dir + "/2_DrimmRaw/blocks.txt"
            sequence = readSequence(block_file)
            drimmSyntenyFile = sp_working_dir + "/2_DrimmRaw/synteny.txt"
            
            list_ = prepare_meta(species_list, genome_meta, chromosome_meta, OG_type, bed_dir)
            sp_list = list_[0]
            sp_ratio = list_[1]
            sp_ratio = ":".join([str(e) for e in sp_ratio]) # convert sp_ratio to a string
            chr_number = list_[2]
            
            drimm_split_blocks_dir,raw_block_dir,result_dir = prepare_dir(sp_working_dir)
            write_raw_blocks(sequence,chr_number,drimm_split_blocks_dir,sp_list)
            processLCSAndFirstFilter = plff.processLCSAndFirstFilter(drimm_split_blocks_dir, raw_block_dir, sp_ratio,
                                                         drimm_split_blocks_dir, homolog_dir, drimmSyntenyFile,
                                                         sp_list, 's')
            processLCSAndFirstFilter.excute()

            processFinalFilter = pff.processFinalFilter(sp_list, raw_block_dir, drimm_split_blocks_dir, result_dir, 's')
            processFinalFilter.excute()
            shutil.rmtree(raw_block_dir)


In [ ]:
# use grimm2barplot to to do chromosome painting
## use R packages chromoMap to paint blocks on full chromosme background (a difference to iage_plot function);
## 2.1 let's first make a combed file based on normal bed, this special bed will be used by combed_parser() and grimmBlock_parser()
beddir = BED_folder
comBed = make_comBed(beddir)
#comBed.to_csv(workdir + 'combBed.txt', sep='\t', header=True, index=False)

## 2.2 let's read in cleaned dirmm blocks, parse it and generate new data <5_SynBlocks>; no need to worry about ratio as all scenarios will be considered in one run
working_dir = workdir + "out1"
if not working_dir.endswith("/"): working_dir += "/"

target = ["sp46"]

for species in target:
    blockDir = working_dir + species + "/3_DrimmBlocks/finalBlocks/"
    output_path = blockDir + "../../5_SynBlocks/"
    if not os.path.exists(output_path): os.mkdir(output_path)
    dic = combed_parser(comBed)
    blockFiles = [os.path.join(blockDir, filename) for filename in os.listdir(blockDir) if filename.endswith('.synteny.genename')]
    grimmBlock_parser(blockFiles,dic,output_path)

In [ ]:
## 2.3 take outputs from last step, generate table for plotting in R package <chromoMap>; no need to worry about ratio as all scenarios will be considered in one run

target = ["sp46"]
reference = "sp03v5"
working_dir = workdir + "out1"
if not working_dir.endswith("/"): working_dir += "/"

for species in target:
    blockDir = working_dir + species + "/5_SynBlocks/"
    blocks = [file for file in os.listdir(blockDir) if file.endswith("synBlock.txt")]
    for block in blocks:
        target_block_name = block.split("_")[0]
        target_block_type = block.split("_")[1]
        reference_block = blockDir + reference + "_" + target_block_type + "_synBlock.txt"
        target_block = os.path.join(blockDir, block)
        target_chr = blockDir + "../3_DrimmBlocks/finalBlocks/" + target_block_name + "_" + target_block_type +".final.block"
        
        dic1,dic2 = get_block_info(reference_block,target_block)
        coo_gene,coo_bp = get_chromosome_info(target_chr,dic1,dic2)

        columns = ["chr","haplotype", "start", "end", "ancestral_group"]
        output_path = blockDir + "../6_chromPainting/"
        if not os.path.exists(output_path): os.mkdir(output_path)

        r1 = pd.DataFrame(coo_gene, columns=columns) # make a dataframe from the list of list
        r2 = pd.DataFrame(coo_bp, columns=columns) # make a dataframe from the list of list
        out_put1 = output_path + target_block_name + "_" + target_block_type +"_cooGene.txt"
        out_put2 = output_path + target_block_name + "_" + target_block_type +"_cooBp.txt"
        r1.to_csv(out_put1, sep='\t', header=True, index=False)
        r2.to_csv(out_put2, sep='\t', header=True, index=False)

In [6]:
## 2.4 generate inputs for chromoMap
sp='sp46'
index_file = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/Smar.EM05.v1.genome.fa.fai"
df_index = make_index(sp,index_file) #your genome index and genome name you want to use
DIC = {f"{row['sp']}@{row['chr']}": row['length'] for _, row in df_index.iterrows()}

genome_meta = make_genome_meta("sp46",1,17)
chromosome_meta = make_chromosome_meta(BED_folder,DIC)

target = ["sp46"]

workdir = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir"
working_dir = workdir + "/out1"
if working_dir[-1] != "/": working_dir += "/"

# obtain information of chromosome number in all species from genome_meta
#genome_meta_df = pd.read_csv(genome_meta, sep='\t', dtype = str, keep_default_na=False, names=['genome', 'ratio1', 'ratio2','chromosome'])
genome_meta_dict = genome_meta.set_index('sp')['chrN'].to_dict()
chromosome_meta_dict = {lst[0]: lst[1:] for lst in chromosome_meta}

for genome in target:
    outdir = working_dir + genome + "/"
    chromosome_list = chromosome_meta_dict[genome]
    chr_info_list = []
    i = 1
    for chromosome in chromosome_list:
        if i <= int(genome_meta_dict[genome]): # make sure only take the long chromosomes as specified in genome_meta
            drimm_chr = "chr_" + chromosome.split("_")[-2]
            genome_chr = "_".join(chromosome.split("_")[0:-2])
            length = chromosome.split("_")[-1]
            chr_info = [drimm_chr,genome_chr,"0",length]
            chr_info_list.append(chr_info)
            i += 1
    df = pd.DataFrame(chr_info_list, columns=['drimm_chr', 'genome_chr', 'start','end'])
    df = df.sort_values(by='genome_chr')
    df.to_csv(outdir+genome+".txt",sep="\t",index=False)